# 리뷰 번역 함수

In [1]:
import json 

with open("./data/GoogleKey.json") as fr:
    google_secret = json.loads(fr.read())

In [2]:
from google.oauth2.service_account import Credentials
from google.cloud import translate

credentials = Credentials.from_service_account_info(google_secret)
google_translate_client = translate.TranslationServiceClient(credentials=credentials)

In [3]:
def get_translation(query):
    parent = f"projects/{google_secret['project_id']}/locations/global"
    response = google_translate_client.translate_text(
        request={
            "parent": parent,
            "contents": [query],
            "mime_type": "text/plain",
            "source_language_code": "ko",
            "target_language_code": "en-US",
        }
    )
    return response.translations[0].translated_text

In [4]:
import csv
import pandas as pd

In [9]:
# 브랜드 별 파일 이름 리스트 
file_names = [
    "ShinSoenAe_신선애 생 닭가슴살 100g~1kg_review"    
]

In [16]:
def translate_review(file):
    file_path = f"./total_preprocessed_review/ShinSeonAe/{file}" + ".csv"
    df = pd.read_csv(file_path, encoding="utf-8")
    for i in range(len(df)):
        review = df.review.iloc[i]
        translated_review = get_translation(review)
        df.loc[i, "translated_review"] = translated_review
    print("translate review 끝")
    return df

In [17]:
def save_file(dataframe, file):
    translated_df = dataframe
    file_path = f"./total_preprocessed_review/ShinSeonAe/{file}" + "_translated.csv"
    translated_df.to_csv(file_path, encoding="utf-8")
    print("file_path", file_path)

In [ ]:
from tqdm import tqdm

def translate_start():
    # tqdm을 사용하여 enumerate를 감싸서 진행 상황을 모니터링
    for i, file in enumerate(tqdm(file_names, desc="Translating files")):
        translated_df = translate_review(file)
        save_file(translated_df, file)
    print("translate start 끝")

translate_start()